In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('operations').getOrCreate()

In [2]:
dataset1 = spark.read.csv('Datasets/traindataset.csv', inferSchema=True, header=True)
dataset2 = spark.read.csv('Datasets/testdataset.csv', inferSchema=True, header=True)
dataset1.printSchema()
dataset2.printSchema()

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)

root
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)



In [3]:
dataset1.show()
dataset2.show()

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+
|30669|  Male| 3.0|           0|            0|          No|     children|         Rural|            95.12|18.0|           null|
|30468|  Male|58.0|           1|            0|         Yes|      Private|         Urban|            87.96|39.2|   never smoked|
|16523|Female| 8.0|           0|            0|          No|      Private|         Urban|           110.89|17.6|           null|
|56543|Female|70.0|           0|            0|         Yes|      Private|         Rural|            69.04|35.9|formerly smoked|
|46136|  Male|14.0|           0|            0|          No| Never_worked|         Rural|           161.2

In [4]:
dataset1.count()

43400

In [5]:
len(dataset1.columns)

11

In [6]:
dataset2.count()

18601

In [7]:
len(dataset2.columns)

11

In [8]:
concat = dataset1.union(dataset2)
concat.show()

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+
|30669|  Male| 3.0|           0|            0|          No|     children|         Rural|            95.12|18.0|           null|
|30468|  Male|58.0|           1|            0|         Yes|      Private|         Urban|            87.96|39.2|   never smoked|
|16523|Female| 8.0|           0|            0|          No|      Private|         Urban|           110.89|17.6|           null|
|56543|Female|70.0|           0|            0|         Yes|      Private|         Rural|            69.04|35.9|formerly smoked|
|46136|  Male|14.0|           0|            0|          No| Never_worked|         Rural|           161.2

In [9]:
concat.count()

62001

In [10]:
len(concat.columns)

11

In [11]:
concat.groupby('gender').count().show()

+------+-----+
|gender|count|
+------+-----+
|Female|36622|
| Other|   13|
|  Male|25366|
+------+-----+



In [12]:
concat.groupby('hypertension').count().show()

+------------+-----+
|hypertension|count|
+------------+-----+
|           1| 5794|
|           0|56207|
+------------+-----+



In [13]:
concat.groupby('work_type').count().show()

+-------------+-----+
|    work_type|count|
+-------------+-----+
| Never_worked|  252|
|Self-employed| 9654|
|      Private|35584|
|     children| 8769|
|     Govt_job| 7742|
+-------------+-----+



In [14]:
concat.groupby('residence_type').count().show()

+--------------+-----+
|residence_type|count|
+--------------+-----+
|         Urban|31066|
|         Rural|30935|
+--------------+-----+



In [15]:
concat.groupby('ever_married').count().show()

+------------+-----+
|ever_married|count|
+------------+-----+
|          No|22124|
|         Yes|39877|
+------------+-----+



In [16]:
concat.groupby('smoking_status').count().show()

+---------------+-----+
| smoking_status|count|
+---------------+-----+
|           null|19043|
|         smokes| 9319|
|   never smoked|22886|
|formerly smoked|10753|
+---------------+-----+



In [17]:
concat.groupby('heart_disease').count().show()

+-------------+-----+
|heart_disease|count|
+-------------+-----+
|            1| 2956|
|            0|59045|
+-------------+-----+



In [18]:
concat.describe().show()

+-------+-----------------+------+------------------+-------------------+-------------------+------------+---------+--------------+-----------------+-----------------+---------------+
|summary|               id|gender|               age|       hypertension|      heart_disease|ever_married|work_type|Residence_type|avg_glucose_level|              bmi| smoking_status|
+-------+-----------------+------+------------------+-------------------+-------------------+------------+---------+--------------+-----------------+-----------------+---------------+
|  count|            62001| 62001|             62001|              62001|              62001|       62001|    62001|         62001|            62001|            59948|          42958|
|   mean|36452.51481427719|  null|42.169475331042975|0.09345010564345736|0.04767665037660683|        null|     null|          null| 104.453831712391| 28.5870988189766|           null|
| stddev|21067.15594488889|  null|22.522099535371346| 0.2910645114143387|0.21308

In [19]:
concat.crosstab('gender', 'heart_disease').show()

+--------------------+-----+----+
|gender_heart_disease|    0|   1|
+--------------------+-----+----+
|               Other|   13|   0|
|                Male|23606|1760|
|              Female|35426|1196|
+--------------------+-----+----+



In [20]:
concat.crosstab('smoking_status', 'heart_disease').show()

+----------------------------+-----+---+
|smoking_status_heart_disease|    0|  1|
+----------------------------+-----+---+
|             formerly smoked| 9818|935|
|                      smokes| 8713|606|
|                never smoked|22024|862|
|                        null|18490|553|
+----------------------------+-----+---+



In [21]:
concat.crosstab('ever_married', 'heart_disease').show()

+--------------------------+-----+----+
|ever_married_heart_disease|    0|   1|
+--------------------------+-----+----+
|                        No|21893| 231|
|                       Yes|37152|2725|
+--------------------------+-----+----+



In [22]:
concat.crosstab('work_type', 'heart_disease').show()

+-----------------------+-----+----+
|work_type_heart_disease|    0|   1|
+-----------------------+-----+----+
|               children| 8765|   4|
|           Never_worked|  251|   1|
|          Self-employed| 8718| 936|
|                Private|33946|1638|
|               Govt_job| 7365| 377|
+-----------------------+-----+----+



In [23]:
concat.crosstab('residence_type', 'heart_disease').show()

+----------------------------+-----+----+
|residence_type_heart_disease|    0|   1|
+----------------------------+-----+----+
|                       Rural|29441|1494|
|                       Urban|29604|1462|
+----------------------------+-----+----+



In [24]:
concat.crosstab('hypertension', 'heart_disease').show()

+--------------------------+-----+----+
|hypertension_heart_disease|    0|   1|
+--------------------------+-----+----+
|                         1| 5040| 754|
|                         0|54005|2202|
+--------------------------+-----+----+



In [25]:
concat.drop('id').columns

['gender',
 'age',
 'hypertension',
 'heart_disease',
 'ever_married',
 'work_type',
 'Residence_type',
 'avg_glucose_level',
 'bmi',
 'smoking_status']

In [26]:
from pyspark.sql.functions import col,sum

In [27]:
concat.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in concat.columns)).show()

+---+------+---+------------+-------------+------------+---------+--------------+-----------------+----+--------------+
| id|gender|age|hypertension|heart_disease|ever_married|work_type|Residence_type|avg_glucose_level| bmi|smoking_status|
+---+------+---+------------+-------------+------------+---------+--------------+-----------------+----+--------------+
|  0|     0|  0|           0|            0|           0|        0|             0|                0|2053|         19043|
+---+------+---+------------+-------------+------------+---------+--------------+-----------------+----+--------------+



In [28]:
concat.dropna().count()

41495

In [29]:
concat.select('gender','age','hypertension','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status').show()

+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+
|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level| bmi| smoking_status|
+------+----+------------+-------------+------------+-------------+--------------+-----------------+----+---------------+
|  Male| 3.0|           0|            0|          No|     children|         Rural|            95.12|18.0|           null|
|  Male|58.0|           1|            0|         Yes|      Private|         Urban|            87.96|39.2|   never smoked|
|Female| 8.0|           0|            0|          No|      Private|         Urban|           110.89|17.6|           null|
|Female|70.0|           0|            0|         Yes|      Private|         Rural|            69.04|35.9|formerly smoked|
|  Male|14.0|           0|            0|          No| Never_worked|         Rural|           161.28|19.1|           null|
|Female|47.0|           